In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate('kiosk-firestore-jnsy-bcfe4-firebase-adminsdk-269q4-4a795774bb.json')
firebase_admin.initialize_app(cred)

In [ ]:
#데이터 전처리까지..

In [ ]:
# 감정 키워드는 원 핫 인코딩으로 데이터 전처리한다.
#from sklearn.preprocessing import OneHotEncoder
#일단은 첫번째감정+두번째감정 으로 저장하는데 가중치 고려할 방법 있었으면 좋겠다
#이 방법은 순서를 중요시하고, or로 합하는 방법은 단순 평균!

# 8가지 감정 키워드
emotion_keyword = ["anger","contempt","disgust","fear","happiness","neutral","sadness","surprise"]

emotion_df = pd.DataFrame({ 'emotion_keyword':emotion_keyword })
emotion_vec = pd.get_dummies(emotion_df).values
#print(emotion_vec)

emotion_keyword_to_vec = {}
for index,keyword in enumerate(emotion_keyword):
    emotion_keyword_to_vec[keyword] = ''.join(list(map(str,emotion_vec[index].tolist())))
    
print(emotion_keyword_to_vec)

In [ ]:
docs = firestore.client().collection(u'order').get()

doc_list = []
for doc in docs:
    #print(doc.get('emotion'))
    #print(type(doc))
    doc_list_row = []
    # 메뉴 - 레이블
    doc_list_row.append(doc.get('orderToString'))
 
    # 날씨
    weather_key_list = []
    for key in doc.get('weather').keys():
        #weather_key_list.append(doc.get('weather').get(key))
        doc_list_row.append(doc.get('weather').get(key))
    
    # 감정
    emotion_key_list = ''
    for key in doc.get('emotion').keys():
        #print(key)
        emotion_key_list += emotion_keyword_to_vec.get(key)
    doc_list_row.append(emotion_key_list)
    
    doc_list.append(doc_list_row)
#print(doc_list)
    

In [ ]:
doc_df = pd.DataFrame(doc_list, columns=['item_name','w_humidity','w_temp','w_speed','emotion'])
doc_df